<a href="https://colab.research.google.com/github/IIVIIIII/2020_Weathering/blob/main/notebook/ccvi_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Filtering Data

In [3]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graph

# New Section

In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("covid_regression").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://raw.githubusercontent.com/IIVIIIII/2020_Weathering/main/resources/ccvi_nonulls.csv"
spark.sparkContext.addFile(url)
ccvi = spark.read.csv(SparkFiles.get("ccvi_nonulls.csv"), sep=",", header=True)

# Show DataFrame
ccvi.show()


+----------+---------+----------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+
|      FIPS|stateName|countyName|               ccvi|             theme1|             theme2|             theme3|            theme4|            theme5|            theme6|             theme7|
+----------+---------+----------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+
|1001020100|  ALABAMA|   Autauga|        0.441972264|0.45833621799999996|         0.37747817|        0.170811065|       0.841616783|        0.96629414|       0.209805276|        0.287591301|
|1001020200|  ALABAMA|   Autauga| 0.7426192490000001|        0.562935368|        0.584587292|        0.683825769|       0.768946675|0.9823988140000001|       0.209805276|        0.427810423|
|1001020300|  ALABAMA|   Autauga|        0.77

In [ ]:
url ="https://data.cdc.gov/api/views/n8mc-b4w4/rows.csv"
spark.sparkContext.addFile(url)
covid = spark.read.csv(SparkFiles.get("rows.csv"), sep=",", header=True)

# Show DataFrame
covid.show()

+----------+---------+---------------+--------------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|    res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|            process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+--------------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-12|       AL|             01|     ST. CLAIR|           26147|            NA|    NA|     NA|       NA|                           null|               

In [ ]:
covid_cases = covid.filter(covid["county_fips_code"] != 'NA')
covid_deaths = covid_cases.filter(covid_cases["death_yn"] == 'Yes')


NameError: ignored

In [ ]:
covid_cases.show()


NameError: ignored

In [ ]:
# Order a DataFrame by ascending values
df.orderBy(df["points"].asc()).show(5)

+-------+--------------------+--------------------+------+-----+--------+------------------+--------------+-------+------------+
|country|         description|         designation|points|price|province|          region_1|      region_2|variety|      winery|
+-------+--------------------+--------------------+------+-----+--------+------------------+--------------+-------+------------+
|     US|The strong aroma ...|                null|  null| null|    null|              null|          null|   null|        null|
|  Italy|This offers gener...|                null|  null| null|    null|              null|          null|   null|        null|
|     US|This shows a deft...|                null|  null| null|    null|              null|          null|   null|        null|
|  Italy|"This nicely stru...| bitter almond an...|  null|   88|      24|Northeastern Italy|Venezia Giulia|   null|Pinot Grigio|
|     US|               Ripe |                null|  null| null|    null|              null|     

In [ ]:
# Order a DataFrame by descending values
df.orderBy(df["points"].desc()).show(5)

+-------+--------------------+--------------------+------+-----+----------+-----------+--------+--------------------+--------------------+
|country|         description|         designation|points|price|  province|   region_1|region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+-----------+--------+--------------------+--------------------+
|     US|This is an absolu...|           IX Estate|    99|  290|California|Napa Valley|    Napa|           Red Blend|              Colgin|
| France|98-100 Barrel sam...|       Barrel sample|    99| null|  Bordeaux|   Pauillac|    null|Bordeaux-style Re...|Ch̢teau Pontet-Canet|
|     US|There are incredi...|Elevation 1147 Es...|    99|  150|California|Napa Valley|    Napa|  Cabernet Sauvignon|        David Arthur|
| France|A magnificent Cha...|Dom P̩rignon Oeno...|    99|  385| Champagne|  Champagne|    null|     Champagne Blend|     Mo��t & Chandon|
|  Italy|Even better than .

In [ ]:
# Import average function
from pyspark.sql.functions import avg
df.select(avg("points")).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [ ]:
# Using filter
df.filter("price<20").show()

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
|  country|         description|         designation|points|price|        province|            region_1|            region_2|             variety|              winery|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
| Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|        Bulgaria|                null|                null|              Mavrud|        Villa Melnik|
|    Spain|Earthy plum and c...|              Amandi|    90|   17|         Galicia|       Ribeira Sacra|                null|             Menc�_a|      Don Bernardino|
|       US|There's a lot to ...|                null|    90|   18|      California|Russian River Valley|              Sonoma|          Chardonnay|            De

In [ ]:
# Filter by price on certain columns
df.filter("price<20").select(['points','country', 'winery','price']).show()

+------+---------+--------------------+-----+
|points|  country|              winery|price|
+------+---------+--------------------+-----+
|    90| Bulgaria|        Villa Melnik|   15|
|    90|    Spain|      Don Bernardino|   17|
|    90|       US|            De Loach|   18|
|    91|       US|   Trinity Vineyards|   19|
|    91| Portugal|Adega Cooperativa...|   15|
|    86|       US|      Belle Ambiance|   10|
|    86| Portugal| Adega de Cantanhede|   12|
|    86|       US|            Parducci|   13|
|    86| Portugal|    Quinta do Portal|   10|
|    86|   France|               Rigal|   14|
|    86|       US|     The Naked Grape|   18|
|    86|   France|   Georges Vigouroux|   15|
|    86|   France|   Georges Vigouroux|   10|
|    86|       US| Martinez & Martinez|   17|
|    86|       US|           Ironstone|   12|
|    86|       US|       Leaping Horse|   10|
|    86|       US|        Kitchen Sink|   13|
|    86| Portugal|  Wines & Winemakers|   12|
|    86|Argentina|              Zo

### Using Python Comparison Operators

In [ ]:
# Same results only this time using python
df.filter(df["price"] < 200).show()

+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|          region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|              Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|    Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|            Oregon| Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
| France|This is the top w...|    

In [ ]:
df.filter( (df["price"] < 200) | (df['points'] > 80) ).show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [ ]:
df.filter(df["country"] == "US").show()

+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|    Oregon|   Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
|     US|This re-named vin...|              Silice|    95|   65|    Or